In [2]:
from textgenrnn import textgenrnn
import pandas
import numpy as np
import rhymes as r
import importlib
importlib.reload(r)
import re

models = []

models.append(textgenrnn('weights/intro_model_weights.hdf5',
                                vocab_path='weights/intro_model_vocab.json',
                                config_path='weights/intro_model_config.json'))

models.append(textgenrnn('weights/chorus_model_weights.hdf5',
                                vocab_path='weights/chorus_model_vocab.json',
                                config_path='weights/chorus_model_config.json'))

models.append(textgenrnn('weights/verse_model_weights.hdf5',
                                vocab_path='weights/verse_model_vocab.json',
                                config_path='weights/verse_model_config.json'))

models.append(textgenrnn('weights/bridge_model_weights.hdf5',
                                vocab_path='weights/bridge_model_vocab.json',
                                config_path='weights/bridge_model_config.json'))

models.append(textgenrnn('weights/outro_model_weights.hdf5',
                                vocab_path='weights/outro_model_vocab.json',
                                config_path='weights/outro_model_config.json'))



In [3]:
#uses model to generate a list of lyrics
def gen_lyrics(model):

    lyrics = model.generate(25, temperature=1.2, max_gen_length=1000, return_as_list=True, prefix="Sunsets")
    
    return lyrics

#Input: A list of generated lines
#Output: dictionary where the keys are the rhymes and the values are a list of lines that rhyme
def post_process(lines):

    rhyme_dict = dict()

    #Generate of list of last words from each line
    for index, line in enumerate(lines):
        if (len(line) == 0):
            continue
        try:
            wordsInLine = line.split()
            curr_word = ""
            wordsInLine[-1] = wordsInLine[-1].lower()
            #remove puncuation
            for c in wordsInLine[-1]:
                if ord(c) >= 97 and ord(c) <= 122:
                    curr_word += c
            #only add words (remove any empty strings)
            if curr_word != "":
                IT_RHYMES_BABYYY = False
                for key in rhyme_dict.keys():
                    if (r.doTheyRhyme(curr_word, key)):
                        rhyme_dict[key].append(index)
                        IT_RHYMES_BABYYY = True
                if (not IT_RHYMES_BABYYY):
                    rhyme_dict[curr_word] = [index]
        except:
            print (curr_word, 'was not found in the dictionary')
            continue

    return rhyme_dict

#Input dictionary of rhymes with their corresponding line number
#Output: sorts each list of lines in the dictionary
def sort_dict(rhymes_dict):
    
    #loop through each key
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        if (length < 2):
            continue
        
        #get a list of each distance between the list of lines
        distances = []
        distances.append(index_list[1] - index_list[0])
        for i in range(1, length - 1):
            distances.append(min(index_list[i] - index_list[i - 1], index_list[i + 1] - index_list[i]))

        distances.append(index_list[length - 1] - index_list[length - 2])

        
        #do some cool sorting stuff where you sort distances and the list of lines in parallel
        distances, index_list = zip(*sorted(zip(distances, index_list)))
        distances, index_list = (list(t) for t in zip(*sorted(zip(distances, index_list))))
        
        #add the sorted list back to the dictionary
        rhymes_dict[key] = index_list
        
#Input dictionary of rhymes with their corresponding line number
#Output: returns a dictionary with rhymes that are one away    
def rhymes_one_apart(rhymes_dict):
    
    removeKeys = []
    
    #loop through each key, keep track of ones to remove
    for key in rhymes_dict.keys():
        #get the list of lines associated with the rhyme
        index_list = rhymes_dict[key]
        length = len(index_list)
        
        if (length < 2):
            removeKeys.append(key)
            continue

        new_list = []
        oneApart = False
        
        #loop through list of rhymes and only keep keys that have rhymes one line apart
        for i in range(1, length - 1):
            if (index_list[i + 1] - index_list[i] == 1 or index_list[i] - index_list[i - 1] == 1):
                oneApart = True
                new_list.append(index_list[i])
                
        #Check front and end cases
        if (index_list[1] - index_list[0] == 1):
            new_list.append(index_list[0])
        if (index_list[length - 1] - index_list[length - 2] == 1):
            new_list.append(index_list[length - 1])
        
        #add new list to key or remove the key if there were no rhymes one apart
        if (oneApart):
            rhymes_dict[key] = new_list
        else:
            removeKeys.append(key)
            
    for key in removeKeys:
        rhymes_dict.pop(key)
        
                


#Uses dictionary of rhyming lines to generate lyrics with a user's rhyming sequence
def out1(lines, rhyme_dictionary):
    print ('What rhyming scheme would you like to use? (A to Z)')
    rhyme_scheme = input()
    if (not re.fullmatch('[A-Z]+', rhyme_scheme)):
        print ('That is not a valid rhyming scheme.')
        exit()

    # count the required number of lines for each letter/section of the rhyming scheme
    rhyme_counts = dict()
    for char in rhyme_scheme:
        if char in rhyme_counts:
            rhyme_counts[char] += 1
        else:
            rhyme_counts[char] = 1
    print (rhyme_counts)

    # map each letter/section of the rhyming scheme to a rhyme in rhyme_dict
    used_rhymes = set()
    letter_to_rhyme = dict()
    for letter in rhyme_counts.keys():
        for rhyme in rhyme_dictionary.keys():
            # if the current rhyme has enough rhymes for the current letter in the rhyming scheme
            # AND the rhyme hasn't been used, use it
            if (len(rhyme_dictionary[rhyme]) >= rhyme_counts[letter] and rhyme not in used_rhymes):
                letter_to_rhyme[letter] = rhyme_dictionary[rhyme]
                used_rhymes.add(rhyme)
                break

    print (letter_to_rhyme)

    # print the results!
    rhyme_indices = dict()
    for char in rhyme_scheme:
        if (char not in rhyme_indices):
            print (lines[letter_to_rhyme[char][0]])
            rhyme_indices[char] = 1
        else:
            print (lines[letter_to_rhyme[char][rhyme_indices[char]]])
            rhyme_indices[char] += 1

In [4]:
model_lines = []

for model in models:
    lines = gen_lyrics(model)
    lines2 = []
    for line in lines:
        split = line.splitlines()
        for l in split:
            if (len(l.split()) > 1):
                lines2.append(l)
                
    model_lines.append(lines2)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:29<00:00,  1.38s/it]


In [5]:
model_dicts = []
for lines in model_lines:
    rhymes_dict = post_process(lines)
    sort_dict(rhymes_dict)
    model_dicts.append(rhymes_dict)

laisse was not found in the dictionary
estamos was not found in the dictionary
[40, 56, 23]
[16, 16, 17]
[41, 47, 1]
[6, 6, 40]
[32, 34, 51, 62, 8]
[2, 2, 11, 11, 24]
[9, 10, 31]
[1, 1, 21]
[20, 42]
[22, 22]
[24, 30, 45, 61]
[6, 6, 15, 16]
[28, 55]
[27, 27]
[52, 59]
[7, 7]
[58, 63]
[5, 5]
dont was not found in the dictionary
itll was not found in the dictionary
shh was not found in the dictionary
thighhighs was not found in the dictionary
timbaland was not found in the dictionary
mg was not found in the dictionary
[6, 65, 128]
[59, 59, 63]
[5, 35, 136]
[30, 30, 101]
[20, 117]
[97, 97]
[67, 68, 15, 19, 113, 119, 26, 134, 87]
[1, 1, 4, 4, 6, 6, 7, 15, 19]
[17, 61]
[44, 44]
[21, 57]
[36, 36]
[25, 82]
[57, 57]
[130, 131, 125]
[1, 1, 5]
[88, 89, 92, 93, 94, 32]
[1, 1, 1, 1, 1, 56]
[34, 42, 118]
[8, 8, 76]
[43, 44, 51, 56, 64, 71, 124]
[1, 1, 5, 5, 7, 7, 53]
[48, 116]
[68, 68]
[52, 66]
[14, 14]
[118, 121, 95]
[3, 3, 23]
[115, 118, 121]
[3, 3, 3]
pra was not found in the dictionary
nigga was 

In [ ]:
#rhymes_one_apart(rhymes_dict)
for rhymes_dict in model_dicts:
    out1(lines2, rhymes_dict)

What rhyming scheme would you like to use? (A to Z)


In [59]:
for line in model_lines[0]:
    print(line)

sunsets tell grow
gucci, i can do it is that?
sunsets stop in the crowd (dance
ultraviolence, you all the pain lights are low
i said baby now (lights on the floor lights lights out lights lights lights lights wreck out love me, dance? let s watch the
swim from me, oh, yeah
im more
oo - da, oo -
oo - - - b oo -
b phone - oh, oh yeah, emotions if babe
this is (true)
to me if that if you got if
sunsets you and i
tonight, home?
you let me rock the shit (yeah
it s never gonna make a bad together, mine
the one who s got time (oh
oh know
take oxygen,, in the wild out, the world
go with the good
sunsets s a man with you?
don t wanna space
bigger, i don t wanna feel
sunsets though i driven
but, i was just it s a, i won
cause you the one that night
can t you, she s
with a
, you really, know
, but even in my bed
sunsets you and i don t even like
it s a beautiful night
definition of insane
think were something on hide
sunsets i can t see what you can?
sunsets when the lights go
out of the other fa

In [8]:
print(model_dicts[0])

{'no': [116, 117]}
